# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*

## Algorithm: Deadlock

In [1]:
import time

In [2]:
from collections import defaultdict
from time import sleep
from threading import Thread, Lock

# Algorithm

In [3]:
%%time
class SharedState:

    def __init__(self, n):
        self._lock = Lock()
        self._state = defaultdict(int)
        self._resources = [Lock() for _ in range(n)]

    def atomic(self, key, value=0):
        with self._lock:
            self._state[key] += value
            return self._state[key]

    def resource(self, i):
        return self._resources[i]

    def kill(self):
        resources = self._resources
        self._resources = None
        for i in resources:
            i.release()

CPU times: user 26 µs, sys: 0 ns, total: 26 µs
Wall time: 28.4 µs


In [4]:
%%time
def worker(pid, state):
    try:
        while True:
            state.atomic('waiting', 1)
            with state.resource(pid):
                state.atomic('waiting', 1)
                with state.resource(pid - 1):
                    state.atomic('waiting', -2)
                    state.atomic('tasks', 1)

    except RuntimeError:
        pass

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


In [5]:
%%time
def deadlock(n):
    state = SharedState(n)

    for i in range(n):
        Thread(target=worker, args=(i, state)).start()

    while state.atomic('waiting') < 2 * n:
        sleep(1)

    print(n, 'workers; deadlock after', state.atomic('tasks'), 'tasks')
    state.kill()

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.3 µs


# Run

In [6]:
%%time
for i in range(1, 10):
    deadlock(10 * i)

10 workers; deadlock after 16558 tasks
20 workers; deadlock after 44839 tasks
30 workers; deadlock after 66586 tasks
40 workers; deadlock after 94362 tasks
50 workers; deadlock after 88691 tasks
60 workers; deadlock after 123179 tasks
70 workers; deadlock after 156276 tasks
80 workers; deadlock after 167545 tasks
90 workers; deadlock after 232806 tasks
CPU times: user 3.43 s, sys: 180 ms, total: 3.61 s
Wall time: 7.69 s


# The End